## Part 2.1

In [179]:
"""Load dependencies"""
import numpy as np
import scipy.sparse as sp
from scipy.sparse import csc_matrix, csc_array, eye
from scipy.sparse.linalg import spsolve, splu
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation

In [180]:
## Given data

eta = 0.21
gamma = 100
alpha = 0.2
w = 0.3

def v(z):
    return 1-4*(z-1/2)**2

In [181]:
M = 1000
dz = 1/M
N = int(round(w/dz))
z = np.linspace(0,1+w,M+N+1)
A1_data = np.array([eta*np.ones(z[1:M].shape)/(v(z[1:M]+dz)*dz**2), -2*eta*np.ones(z[1:M].shape)/(v(z[1:M])*dz**2), eta*np.ones(z[1:M].shape)/(v(z[1:M]-dz)*dz**2)])
A1 = sp.spdiags(A1_data, np.array([-1, 0, 1]), format='csc')

# Adjust for first boundary condition:
A1[0,0] = eta/(v(z[1])*dz**2)*(-2/3); A1[0,1] = eta/(v(z[1])*dz**2)*(2/3)

A2_data = np.array([np.ones(z[M+1:M+N].shape)/(dz**2), -2*np.ones(z[M+1:M+N].shape)/(dz**2)-gamma, np.ones(z[M+1:M+N].shape)/(dz**2)])
A2 = sp.spdiags(A2_data, np.array([-1, 0, 1]), format='csc'); A2[-1,-1] = 1/(dz**2)*(-2/3)-gamma; A2[-1,-2] = 1/(dz**2)*(2/3)

b1 = np.zeros((A1.shape[0],1)); b1[-1] = -1/dz
b2 = np.zeros((A2.shape[0],1)); b2[0] = -alpha/dz
a = (1+alpha)/dz
e1 = np.zeros((A1.shape[0],1)); e1[-1] = eta/((dz**2)*v((M-1)*dz))
e2 = np.zeros((A2.shape[0],1)); e2[0] = 1/(dz**2)

/var/folders/m9/hb_z9t0x7tj7c1c__gblxfc00000gn/T/ipykernel_42300/3399971855.py:5: RuntimeWarning: divide by zero encountered in divide
  A1_data = np.array([eta*np.ones(z[1:M].shape)/(v(z[1:M]+dz)*dz**2), -2*eta*np.ones(z[1:M].shape)/(v(z[1:M])*dz**2), eta*np.ones(z[1:M].shape)/(v(z[1:M]-dz)*dz**2)])


In [182]:
#Concatenate everything to block matrix system
block1 = sp.hstack([A1,csc_matrix(e1),csc_matrix(np.zeros((A1.shape[0], A2.shape[1])))])
block2 = sp.hstack([csc_matrix(b1.T), csc_matrix(a), csc_matrix(b2.T)])
block3 = sp.hstack([csc_matrix(np.zeros((A2.shape[0], A1.shape[1]))), csc_matrix(e2), A2])
A = sp.vstack([block1,block2,block3])

In [185]:
I_A1 = np.eye(A1.shape[0])
I_tot = np.zeros(A.shape)
# Insert I_A1 to I_tot
I_tot[:I_A1.shape[0], :I_A1.shape[0]] = I_A1

I_tot = csc_matrix(I_tot)
dtau = 0.0001

# Initial condition
u0 = np.zeros((A.shape[1],1)); u0[:M-1] = 1

## Implicit Euler
tau = np.arange(dtau, 1, dtau)
saved_u = np.zeros((len(u0), len(tau)))
saved_u[:, 0] = u0[:, 0]

uk = u0

LHS = splu(I_tot - dtau*A)
for i,_ in enumerate(tau):
    RHS = I_tot.dot(uk)
    u_new = LHS.solve(RHS)
    #u_new = spsolve(LHS, RHS, use_umfpack=False)
    saved_u[:, i] = u_new[:,0]
    uk = u_new

In [186]:
""" 3D plot """
%matplotlib qt

Z, TAU = np.meshgrid(z[1:-1],tau)

fig = plt.figure() 
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(Z, TAU, saved_u.T)


In [178]:
"""Animation"""

fig, ax = plt.subplots()

line, = ax.plot(z[1:-1], saved_u[:, 0])

def update(frame):
    line.set_ydata(saved_u[:, frame])
    return line,

ani = FuncAnimation(fig, update, frames=range(saved_u.shape[1]), interval=1, blit=True)